# PyOpenCAP tutorial
In this tutorial we, post process results from Q-Chem calculations to analyze eigenvalue trajectories for CAP/EOM-CC and CAP-ADC calculations on the ${}^2\Pi_g$ shape resonance of $N_2^-$. We use the pyopencap.analysis submodule to extract the matrices and analyze the trajectories.

In [ ]:
from pyopencap.analysis import CAPHamiltonian
import matplotlib.pyplot as plt
import numpy as np

# EOM-CC

## Option 1: Analysis of .fchk files

For EOM-CC calculations, one particle densities can be exported to .fchk using GUI=2, CC_TRANS_PROP=3 and STATE_ANALYSIS=1(see examples/opencap/qc_input.in). The snippet below can be used to post-process the outputs generated by Q-Chem in order to obtain the zeroth order Hamiltonian and the CAP matrix.

In [ ]:
'''
sys_dict = {"molecule":  "qchem_fchk",
            "basis_file": "n2.fchk"
}

cap_dict = {
            "cap_type": "box",
            "cap_x":"2.76",
            "cap_y":"2.76",
            "cap_z":"4.88"
}

es_dict = { "package": "qchem",
            "method" : "eom",
           "qchem_output":"n2.out",
           "qchem_fchk":"n2.fchk",
}

s = pyopencap.System(sys_dict)
pc = pyopencap.CAP(s,cap_dict,5)
pc.read_data(es_dict)
pc.compute_ao_cap()
pc.compute_projected_cap()
W = pc.get_projected_cap()
H0 = pc.get_H()
'''

## Option 2: Post-processing Projected CAP-EOM-CC Q-Chem outputs
Projected CAP-EOM-CC is implemented in Q-Chem (available as of Q-Chem 5.4), and can be requested using the PROJ_CAP=3 keyword in complex_ccman. PyOpenCAP is capable of performing analysis directly from outputs generated by this calculation. 

In [ ]:
from pyopencap.analysis import CAPHamiltonian
import matplotlib.pyplot as plt
import numpy as np
# Q-Chem output, can specify irrep
CAPH = CAPHamiltonian(qchem_output="ref_outputs/eomcc.out",irrep="B2g")
# OpenCAP output
CAPH = CAPHamiltonian(opencap_output="ref_outputs/n2_opencap.out")

In [ ]:
eta_list = np.linspace(0,3000,101)
eta_list = np.around(eta_list * 1E-5,decimals=5)
CAPH.run_trajectory(eta_list)

In [ ]:
# ccsd energy of neutral
ref_energy = -109.36195558
traj = CAPH.track_state(1,tracking="overlap")
uc_energies = traj.energies_ev(ref_energy=ref_energy)
corr_energies = traj.energies_ev(ref_energy=ref_energy,corrected=True)
plt.plot(np.real(uc_energies),np.imag(uc_energies),'-ro',label="Uncorrected")
plt.plot(np.real(corr_energies),np.imag(corr_energies),'-bo',label="Corrected")
plt.legend()
plt.show()

In [ ]:
# Find optimal value of eta
uc_energy, eta_opt = traj.find_eta_opt(start_idx=10)
uc_energy = (uc_energy-ref_energy)*27.2114
corr_energy, corr_eta_opt = traj.find_eta_opt(corrected=True,start_idx=10)
corr_energy = (corr_energy-ref_energy)*27.2114
print("Uncorrected:")
print(uc_energy)
print(eta_opt)
print("Corrected:")
print(corr_energy)
print(corr_eta_opt)
plt.plot(np.real(uc_energies),np.imag(uc_energies),'-ro',label="Uncorrected")
plt.plot(np.real(corr_energies),np.imag(corr_energies),'-bo',label="Corrected")
plt.plot(np.real(uc_energy),np.imag(uc_energy),'g*',markersize=20)
plt.plot(np.real(corr_energy),np.imag(corr_energy),'g*',markersize=20)
plt.legend()
plt.show()

## ADC

Starting from Q-Chem 5.4, CAP-ADC calculations can be performed using the ADC_CAP keyword. PyOpenCAP is capable of performing analysis directly from outputs generated by this calculation, either in the form of .data files or the Q-Chem output file. Here, we a analyze CAP-EA-ADC(2) calculation.

In [ ]:
# Q-Chem output, can specify onset
CAPH = CAPHamiltonian(qchem_output="ref_outputs/adc.out", onset="3000")

In [ ]:
eta_list = np.linspace(0,3000,101)
eta_list = np.around(eta_list * 1E-5,decimals=5)
CAPH.run_trajectory(eta_list)

In [ ]:
# energies in hamiltonian are excitation energies rather than total energies, so ref_energy=0 is appropriate ehre
traj = CAPH.track_state(1,tracking="overlap")
uc_energies = traj.energies_ev(ref_energy=0.0)
corr_energies = traj.energies_ev(ref_energy=0.0,corrected=True)
plt.plot(np.real(uc_energies),np.imag(uc_energies),'-ro',label="Uncorrected")
plt.plot(np.real(corr_energies),np.imag(corr_energies),'-bo',label="Corrected")
plt.legend()
plt.show()

In [ ]:
# Find optimal value of eta
uc_energy, eta_opt = traj.find_eta_opt(start_idx=10)
uc_energy = uc_energy*27.2114
corr_energy, corr_eta_opt = traj.find_eta_opt(corrected=True,start_idx=10)
corr_energy = corr_energy*27.2114
print("Uncorrected:")
print(uc_energy)
print(eta_opt)
print("Corrected:")
print(corr_energy)
print(corr_eta_opt)
plt.plot(np.real(uc_energies),np.imag(uc_energies),'-ro',label="Uncorrected")
plt.plot(np.real(corr_energies),np.imag(corr_energies),'-bo',label="Corrected")
plt.plot(np.real(uc_energy),np.imag(uc_energy),'g*',markersize=20)
plt.plot(np.real(corr_energy),np.imag(corr_energy),'g*',markersize=20)
plt.legend()
plt.show()